In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.colors as colors
import matplotlib.pyplot as plt

def getHeatMap(newdataf,month,filepath,count,boolval):
    #Filtraggio per mese
    newdf=newdataf[newdataf['hours'].str.contains(month)]
    #Conta numero di osservazioni in ogni ora
    newdf=newdf.groupby(['hours'],sort=False,as_index=False)[['count']].count()
    #Estrai giorno e ora
    newdf['day']=newdf['hours'].str[:2]
    newdf['h']=newdf['hours'].str[-3:-1]
    #Crea nuovo daframe per heatmap
    f=pd.DataFrame(columns=['h','count'])
    f['day']=newdf['day']
    f['h']=newdf['h'].astype(int)
    f['count']=newdf['count'].astype(int)
    #Crea una pivot table
    piv = pd.pivot_table(f, values="count",index=["h"], columns=["day"], fill_value=0)
    # This dictionary defines the colormap
    cdict = {'red':  ((0.0, 0.8, 0.8),   # no red at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.0, 0.0)),  # set to 0.8 so its not too bright at 1

        'green': ((0.0, 0.0, 0.0),   # set to 0.8 so its not too bright at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.8, 0.8)),  # no green at 1

        'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.0, 0.0))   # no blue at 1
       }
    #Creazione heatmap con grana 1 secondo
    GnRd = colors.LinearSegmentedColormap('GnRd', cdict)
    ax = sns.heatmap(piv, mask=piv==0,square=True,linewidths=0.8,cmap=GnRd,vmin=0,vmax=3600)    
    figure = ax.get_figure() 
    heatDir=os.path.join(os.path.abspath('..'),os.path.join("heatmap\\","EB"+str(count)+"_"+str(3600)+"\\"+filepath[:-4]))
    if not os.path.exists(heatDir):
        os.makedirs(heatDir)
    heatmapPath=os.path.join(heatDir,month+'.png')
    figure.savefig(heatmapPath, dpi=400)
    plt.clf()
    #Crea heatmap con grana 5 secondi per meter
    GnRd = colors.LinearSegmentedColormap('GnRd', cdict)
    if boolval==False:
        ax2 = sns.heatmap(piv,mask=piv==0, square=True,linewidths=0.8,cmap=GnRd,vmin=0,vmax=720)
        figure2 = ax2.get_figure() 
        heatDir=os.path.join(os.path.abspath('..'),os.path.join("heatmap\\","EB"+str(count)+"_"+str(720)+"\\"+filepath[:-4]))
        if not os.path.exists(heatDir):
            os.makedirs(heatDir)
        heatmapPath=os.path.join(heatDir,month+'.png')
        figure2.savefig(heatmapPath, dpi=400)
    plt.clf()
    
    
    
#Funzione che ritorna dataframe in base al mese   
def switch_demo(argument):
    global jandf
    global febdf
    global mardf
    global aprdf
    global maydf
    global jundf
    if(argument=='Jan'):
        return jandf
    if(argument=='Feb'):
        return febdf
    if(argument=='Mar'):
        return mardf
    if(argument=='Apr'):
        return aprdf
    if(argument=='May'):
        return maydf
    if(argument=='Jun'):
        return jundf
#Funzione che assegna le modifiche al dataframe in base al mese
def switch_df(argument,df):
    global jandf
    global febdf
    global mardf
    global aprdf
    global maydf
    global jundf
    if(argument=='Jan'):
        jandf=df
    if(argument=='Feb'):
        febdf=df
    if(argument=='Mar'):
        mardf=df
    if(argument=='Apr'):
        aprdf=df
    if(argument=='May'):
        maydf=df
    if(argument=='Jun'):
        jundf=df

def process(chunck,filepath,count):
    global jandf
    global febdf
    global mardf
    global aprdf
    global maydf
    global jundf
    df=chunck.loc[:, ('timestamp', 'power')]
    # generiamo la data nel formato corretto a partire dal timestamp unix
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize("GMT").dt.tz_convert('Europe/Rome')
    # arrotondiamo da ms al secondo pi� vicino
    df['date'] = df['date'].dt.round('1s')
    df = df[['date', 'power']]
    # crea una time series settando la data come index
    df = df.set_index('date')
    #Raggruppa per data ed effettua la media dei power
    df=df.groupby('date')[['power']].mean()
    #Aggiungi una colonna contenente  data e ora per heatmap
    df['hours']=df.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    df['day']=df.index.to_series(keep_tz=True).dt.strftime("%d").str.lstrip("0").replace(" 0", " ")
    df['count']=0
    monthList=['Jan','Feb','Mar','Apr','May','Jun']
    #Esegui i vari filtraggi e crea un dataframe contenente i valori per ogni mese
    for month in monthList:
        newdf=df[df['hours'].str.contains(month)]
        filterdf=switch_demo(month)
        frames = [filterdf,newdf]
        filterdf=pd.concat(frames)
        switch_df(month,filterdf)
 

In [6]:
DIRPATH=["\Dati puntuali EB1 Gennaio-Giugno 2019","\Dati puntuali EB2 Gennaio-Giugno 2019","\Dati puntuali EB3 Gennaio-Giugno 2019","\Dati puntuali EB4 Gennaio-Giugno 2019","\Dati puntuali EB5 Gennaio-Giugno 2019","\Dati puntuali EB6 Gennaio-Giugno 2019","\Dati puntuali EB7 Gennaio-Giugno 2019","\Dati puntuali EB8 Gennaio-Giugno 2019","\Dati puntuali EB9 Gennaio-Giugno 2019","\Dati puntuali EB10 Gennaio-Giugno 2019"]
#DIRPATH=["\Dati puntuali EB1 Gennaio-Giugno 2019"]
count=0
for dirpath in DIRPATH:
    count=count+1
    print("Start "+str(count))
    dirp=os.path.join(os.path.abspath('..'),"data"+dirpath+"\\")
    filelist=[f for f in os.listdir(dirp)]
    for filepath in filelist:
        dataPath=os.path.join(dirp,filepath)
        #Creazione dataframe vuoti per ogni mese
        jandf=febdf=mardf=aprdf=maydf=jundf=pd.DataFrame(columns=['date','power','hours','day','count'])
        jandf = jandf.set_index('date')
        febdf = febdf.set_index('date')
        mardf = mardf.set_index('date')
        aprdf = aprdf.set_index('date')
        maydf = maydf.set_index('date')
        jundf = jundf.set_index('date')
        pd.set_option('display.max_rows', 10000)
        for chunck in pd.read_csv(dataPath, sep=';',chunksize=500000):
            process(chunck,filepath,count)
        if("Meter" in filepath):
            boolval=False
        else:
            boolval=True
        
        #Creazione heatmap
        getHeatMap(jandf,'Jan',filepath,count,boolval) 
        getHeatMap(febdf,'Feb',filepath,count,boolval)
        getHeatMap(mardf,'Mar',filepath,count,boolval)
        getHeatMap(aprdf,'Apr',filepath,count,boolval)
        getHeatMap(maydf,'May',filepath,count,boolval)
        getHeatMap(jundf,'Jun',filepath,count,boolval)

Start 1


c:\users\giuse\pycharmprojects\untitled\venv\lib\site-packages\seaborn\matrix.py:287: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))
c:\users\giuse\pycharmprojects\untitled\venv\lib\site-packages\seaborn\matrix.py:287: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))


Start 2
Start 3
Start 4
Start 5
Start 6
Start 7
Start 8
Start 9
Start 10


<Figure size 432x288 with 0 Axes>